In [29]:
import json
import pandas as pd
import numpy as np
import time
import requests
import logging
from collections import defaultdict

In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("economic-test")
logger.setLevel(logging.INFO)

In [42]:
def get_information(method, params):
#     url = 'https://api.s0.os.hmny.io/'
    url = 'https://api.s0.t.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [18]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [19]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [20]:
def getCurrentAndLastBlock():
    block = getBlockNumber()
    last_block = getLastBlockOfCurrentEpoch()
    print("current and last block numbers", block, last_block)
    return block, last_block

In [39]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result'] 

In [15]:
def getAllValidatorAddress():
    method = 'hmy_getAllValidatorAddresses'
    params = []
    return get_information(method, params)['result'] 

In [10]:
def getAllValidatorInformationByBlockNumber(block):
    method = 'hmy_getAllValidatorInformationByBlockNumber'
    params = [-1, hex(block)]
    return get_information(method, params)

In [64]:
def getValidatorInfo(validator):
    method = "hmy_getValidatorInformation"
    params = [validator]
    return get_information(method, params)['result']

In [160]:
def getTransactionsCount(shard, address) -> int:
    url = endpoint[shard]
    method = 'hmy_getTransactionsCount'
    params = [address, 'ALL']
    return get_information(url, method, params)

In [169]:
def getTransactionCount(shard, address) -> int:
    url = endpoint[shard]
    method = 'hmy_getTransactionCount'
    params = [address, 'latest']
    return get_information(url, method, params)

In [162]:
address = 'one19wmuswcxr6090haljtr4q63n8x2dwcs60llp5e'
getTransactionsCount(0, address)

{'jsonrpc': '2.0', 'id': 1, 'result': 3}

In [170]:
int(getTransactionCount(0, address)['result'],16)

1

In [82]:
int(getBalanceByBlockNumber(address, block),16)/1e18

14163852.751404636

In [83]:
def get_information(url, method, params) -> dict:
    headers = {'Content-Type': 'application/json; charset=utf8'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    try:
        r = requests.post(url, headers=headers, data = json.dumps(data))
    except requests.exceptions.ConnectionError as e:
        print("Error: connection error")
        time.sleep(5)
        return None
    if r.status_code != 200:
        print("Error: Return status code %s" % r.status_code)
        return None
    try:
        r.json()
    except ValueError:
        print("Error: Unable to read JSON reply")
        return None
    return r.json()

In [139]:
def getBlockByNumber(shard, number) -> dict:
    url = endpoint[shard]
    method = 'hmy_getBlockByNumber'
    params = [hex(number), False]
    return get_information(url, method, params)

In [145]:
import pyhmy 
from pyhmy import rpc

In [155]:
endpoint = ['http://35.165.160.63:9500','http://18.202.18.118:9500','http://3.90.56.193:9500','http://54.64.140.205:9500']

In [173]:
address = 'one14tdlgysvnqcdgwnduttd0y5pp2y7m8cpss30j4'
shard = 1
block_num = {0:3375104, 1:3286736, 2:3326152, 3:3313571}
rpc.account.get_balance_by_block( address, block_num[shard],endpoint[shard])

0

In [87]:
from datetime import datetime

In [149]:
time = int(getBlockByNumber(0, 3391488)['timestamp'],16)
datetime.fromtimestamp(time).strftime('%Y_%m_%d_%H:%M:%S')

'2020_05_18_13:18:48'

In [93]:
endpoint = ['https://api.s0.t.hmny.io/', 'https://api.s1.t.hmny.io/', 'https://api.s2.t.hmny.io/', 'https://api.s3.t.hmny.io/']
time = int(getBlockByNumber(0, 3375104)['timestamp'],16)

In [94]:
datetime.fromtimestamp(time).strftime('%Y_%m_%d_%H:%M:%S')

'2020_05_16_15:08:52'

In [116]:
time = int(getBlockByNumber(1, 3286736)['timestamp'],16)
datetime.fromtimestamp(time).strftime('%Y_%m_%d_%H:%M:%S')

'2020_05_16_15:08:53'

In [127]:
time = int(getBlockByNumber(2, 3326152)['timestamp'],16)
datetime.fromtimestamp(time).strftime('%Y_%m_%d_%H:%M:%S')

'2020_05_16_15:08:56'

In [138]:
time = int(getBlockByNumber(3, 3313571)['timestamp'],16)
datetime.fromtimestamp(time).strftime('%Y_%m_%d_%H:%M:%S')

'2020_05_16_15:08:53'

In [47]:
from collections import Counter
block, last_block = getCurrentAndLastBlock()
while block < last_block:
    block = getBlockNumber()
print("last block in this epoch reached")
validator_info = getAllValidatorInformation()
time.sleep(5)
block = getBlockNumber()
print(f"current block number {block}")
new_validator_info = getAllValidatorInformation()
num = 10
iterations = 0
while iterations < num:
    new_block = block +1
    while block < new_block:
        block = getBlockNumber()
    info = getAllValidatorInformation()
    status = []
    for i in validator_info:
        status.append(i['epos-status'])
    print(f"current block {block}, status: {Counter(status)}")
    iterations += 1
    

current and last block numbers 3417960 3424255
last block in this epoch reached
current block number 3424255


In [48]:
validator_info

[{'validator': {'bls-public-keys': ['15dad6a28f96880c1462e9afa7745a9e65fdc971c981813847a0e4d00849b476c18c1ae9b1d75126b158eeeb21f08910'],
   'last-epoch-in-committee': 188,
   'min-self-delegation': 10000000000000000000000,
   'max-total-delegation': 199999999999999999999999000000000,
   'rate': '0.001000000000000000',
   'max-rate': '1.000000000000000000',
   'max-change-rate': '1.000000000000000000',
   'update-height': 3374860,
   'name': 'CONTABO-1#PLEASE DELEGATE to ME or to VALIDATORS THAT NEED HELP2GET ELECTED#SEARCH 4 NOT-ELECTED WITH Best UPTIME#TY#LetsProveThatEPoSworks#',
   'identity': 'Contabo-1',
   'website': 'www.contabo.com',
   'security-contact': 'Contabo',
   'details': 'Contabo',
   'creation-height': 3358745,
   'address': 'one1hwe68yprkhp5sqq5u7sm9uqu8jxz87fd7ffex7',
   'delegations': [{'delegator-address': 'one1hwe68yprkhp5sqq5u7sm9uqu8jxz87fd7ffex7',
     'amount': 509496000000000000000000,
     'reward': 3039718028432257176912,
     'undelegations': []},
    {'

In [54]:
status = []
for i in validator_info:
    status.append(i['epos-status'])

In [57]:
Counter(status)

Counter({'currently elected': 66,
         'eligible to be elected next epoch': 53,
         'not eligible to be elected next epoch': 51})

In [52]:
new_status = []
for i in new_validator_info:
    new_status.append(i['epos-status'])

In [58]:
Counter(new_status)

Counter({'currently elected': 66,
         'eligible to be elected next epoch': 53,
         'not eligible to be elected next epoch': 51})

In [60]:
curr = getAllValidatorInformation()

In [61]:
curr_status = []
for i in curr:
    curr_status.append(i['epos-status'])

In [62]:
Counter(curr_status)

Counter({'eligible to be elected next epoch': 68,
         'currently elected': 43,
         'not eligible to be elected next epoch': 59})